# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00


In [2]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [3]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-lus18o4a
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-lus18o4a
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit 0ab4688e84181b78317b8433eca7e3aaf70c8a1b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━

In [4]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-02-04 18:22:19--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23304 (23K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  22.76K  --.-KB/s    in 0.001s  

2025-02-04 18:22:20 (30.1 MB/s) - ‘structured_qa.csv’ saved [23304/23304]



# Setup

In [5]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [6]:
from loguru import logger

In [7]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process a single Document

In [12]:
ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        question = row["question"]
        logger.info(f"Question: {question}")
        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(
                    CURRENT_INFO=load_pdf(document_file)
                ),
            },
            {"role": "user", "content": question},
        ]
        try:
            answer = model.get_response(messages)
        except Exception as e:
            answers = {
                index: "Out of context" for index in document_data.index
            }
            sections = {index: None for index in document_data.index}
            return answers, sections
        logger.info(f"Answer: {answer}")
        answers[index] = answer
        sections[index] = None

    return answers, sections

## Load Model

In [9]:
from structured_qa.model_loaders import load_llama_cpp_model

In [10]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2.5-7B-Instruct-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

# Run Benchmark

In [13]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 18:29:09.059 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-02-04 18:29:09.068 | INFO     | __main__:<cell line: 0>:12 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-02-04 18:29:09.069 | INFO     | __main__:<cell line: 0>:18 - File HAI_AI-Index-Report-2024.pdf.pdf already exists
2025-02-04 18:29:09.070 | INFO     | __main__:process_document:24 - Predicting
2025-02-04 18:29:09.073 | INFO     | __main__:process_document:29 - Question: which type of risk was identified as the leading concern globally? -A: Fairness risks. -B: Privacy and data governance risks. -C: Risks related to generative AI deployment.
2025-02-04 18:29:49.358 | INFO     | __main__:<cell line: 0>:12 - Downloading document https://arxiv.org/pdf/1706.03762
2025-02-04 18:29:49.495 | INFO     | __main__:<cell line: 0>:16 - Downloaded https://arxiv.org/pdf/1706.03762 to 1706.03762.pdf
2025-02-04 18:29:49.496 | INFO     | __main__:p

In [14]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
10,10,https://arxiv.org/pdf/1706.03762,Scientific Paper,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,YES,NaN
17,17,https://arxiv.org/pdf/2106.09685.pdf,Scientific Report,4 OUR METHOD,Does LoRA work with any neural network contain...,YES,NO,NaN
22,22,https://authorsalliance.org/wp-content/uploads...,Techincal Documentation,HOW DO YOU CHOOSE AN OPEN ACCESS PUBLISHER?,how many peer-reviewed open access journals ar...,A,B,NaN
24,24,https://authorsalliance.org/wp-content/uploads...,Techincal Documentation,OVERCOMING RESERVATIONS ABOUT OPEN ACCESS,Are publication fees required for all open acc...,NO,I NEED MORE INFO,NaN
27,27,https://arxiv.org/pdf/2201.11903,Scientific Report,3 Arithmetic Reasoning,Is Arithmetic reasoning is a task that languag...,NO,OUT OF CONTEXT,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,OUT OF CONTEXT,NaN
29,29,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many benchmarks were used to evaluate arit...,5,OUT OF CONTEXT,NaN
30,30,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,Is symbolic reasoning usually simple for human...,YES,OUT OF CONTEXT,NaN
31,31,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,How many words have the example names that the...,B,OUT OF CONTEXT,NaN
32,32,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,Which symbolic reasoning task is used as an ou...,A,OUT OF CONTEXT,NaN


In [15]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.5339805825242718